In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import pandas as pd

In [ ]:
path = '/content/gdrive/MyDrive/reforecast'
datasets_path = path+'/dataset/solar_stations'

In [ ]:
files = [os.path.join(path+'/lag_added_dataset', file) for file in os.listdir(path+'/lag_added_dataset') if file.endswith('.csv')]
origin_df_list = []
files.sort()
for file in files:
  print(file)
  df = pd.read_csv(file)

  df = df[[col for col in df.columns if col != 'Direct normal irradiance (W/m2)']]
  df = df[[col for col in df.columns if col != 'Global horizontal irradiance (W/m2)']]
  df = df[[col for col in df.columns if col != 'day']]
  df = df[[col for col in df.columns if col != 'day_sin']]
  df = df[[col for col in df.columns if col != 'day_cos']]
  df = df[[col for col in df.columns if col != 'hour']]
  df = df[[col for col in df.columns if col != 'month']]
  df = df[[col for col in df.columns if col != 'Time']]
  df = df[[col for col in df.columns if col != 'Unnamed: 0.1']]
  df = df[[col for col in df.columns if col != 'Unnamed: 0']]
  origin_df_list.append(df)



print(len(origin_df_list))


/content/gdrive/MyDrive/reforecast/lag_added_dataset/feature_added_dataset_Solar station site 1 (Nominal capacity-50MW).csv
/content/gdrive/MyDrive/reforecast/lag_added_dataset/feature_added_dataset_Solar station site 2 (Nominal capacity-130MW).csv
/content/gdrive/MyDrive/reforecast/lag_added_dataset/feature_added_dataset_Solar station site 4 (Nominal capacity-130MW).csv
/content/gdrive/MyDrive/reforecast/lag_added_dataset/feature_added_dataset_Solar station site 5 (Nominal capacity-110MW).csv
/content/gdrive/MyDrive/reforecast/lag_added_dataset/feature_added_dataset_Solar station site 6 (Nominal capacity-35MW).csv
/content/gdrive/MyDrive/reforecast/lag_added_dataset/feature_added_dataset_Solar station site 7 (Nominal capacity-30MW).csv
/content/gdrive/MyDrive/reforecast/lag_added_dataset/feature_added_dataset_Solar station site 8 (Nominal capacity-30MW).csv
7


In [ ]:
origin_df_list[0].columns

Index(['Time(year-month-day h:m:s)', 'Total solar irradiance (W/m2)',
       'Air temperature  (°C) ', 'Atmosphere (hpa)', 'Power (MW)', 'hour_sin',
       'hour_cos', 'month_sin', 'month_cos', 'same_time_mean', 'same_time_std',
       'same_time_max', 'same_time_min', 'same_weekday_mean',
       'same_weekday_std', 'same_weekday_max', 'same_weekday_min',
       'overall_mean', 'overall_std', 'overall_max', 'overall_min', 'lag_1',
       'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8',
       'diff_15min', 'diff_30min', 'diff_45min', 'diff_60min', 'diff_75min',
       'diff_90min', 'diff_105min'],
      dtype='object')

In [ ]:
origin_df_list[0].head()

,Time(year-month-day h:m:s),Total solar irradiance (W/m2),Air temperature (°C),Atmosphere (hpa),Power (MW),hour_sin,hour_cos,month_sin,month_cos,same_time_mean,...,lag_6,lag_7,lag_8,diff_15min,diff_30min,diff_45min,diff_60min,diff_75min,diff_90min,diff_105min
0,2019-01-08 02:00:00,0,-10.3,925.4,0.0,0.500000,0.866025,0.5,0.866025,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-08 02:15:00,0,-12.0,925.3,0.0,0.500000,0.866025,0.5,0.866025,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-08 02:30:00,0,-11.3,925.3,0.0,0.500000,0.866025,0.5,0.866025,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-08 02:45:00,0,-11.9,925.2,0.0,0.500000,0.866025,0.5,0.866025,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-08 03:00:00,0,-12.1,925.2,0.0,0.707107,0.707107,0.5,0.866025,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
a = [1,2,4,5,6,7,8]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import math

test_size = 2/24
features = ['Total solar irradiance (W/m2)',
       'Air temperature  (°C) ', 'Atmosphere (hpa)', 'Relative humidity (%)',
       'Power (MW)', 'hour_sin', 'hour_cos', 'month_sin',
       'month_cos', 'same_time_mean', 'same_time_std', 'same_time_max',
       'same_time_min', 'same_weekday_mean', 'same_weekday_std',
       'same_weekday_max', 'same_weekday_min', 'overall_mean', 'overall_std',
       'overall_max', 'overall_min', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
       'lag_5', 'lag_6', 'lag_7', 'lag_8', 'diff_15min', 'diff_30min',
       'diff_45min', 'diff_60min', 'diff_75min', 'diff_90min', 'diff_105min']



target = 'Power (MW)'
shifted_column = 'power_shifted'

In [ ]:
import joblib
import numpy as np
from datetime import time
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

result_for_reforecast_df_list = []
a = [1, 2, 4, 5, 6, 7, 8]
forecasted_df_list = []

for i in range(len(origin_df_list)):
  print("===============================================================")
  print(a[i])
  df = origin_df_list[i].copy()

  # 다음 시점 예측을 위한 타겟 시프트
  df[shifted_column] = df[target].shift(-1)
  df = df.dropna().reset_index(drop=True)

  features = [col for col in features if col in df.columns]
  X = df[features]
  y = df[shifted_column]
  time_col = df['Time(year-month-day h:m:s)']

  # 시계열 순서 기반 분할
  split_index = int(len(X) * (1 - test_size))
  X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
  y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
  time_train, time_test = time_col.iloc[:split_index], time_col.iloc[split_index:]

  # 정규화
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  # 모델 학습
  print("MLR")
  mlr = LinearRegression()
  mlr.fit(X_train_scaled, y_train)

  print("SVR")
  svr = SVR()
  svr.fit(X_train_scaled, y_train)

  print("LightGBM")
  lgb_model = lgb.LGBMRegressor()
  lgb_model.fit(X_train, y_train)

  print("MLP")
  mlp = MLPRegressor(random_state=42, max_iter=1000)
  mlp.fit(X_train_scaled, y_train)

  # 예측 결과 저장
  df_result = X.copy()
  df_result.loc[:split_index-1, 'pred_mlr'] = mlr.predict(X_train_scaled)
  df_result.loc[split_index:, 'pred_mlr'] = mlr.predict(X_test_scaled)

  df_result.loc[:split_index-1, 'pred_svr'] = svr.predict(X_train_scaled)
  df_result.loc[split_index:, 'pred_svr'] = svr.predict(X_test_scaled)

  df_result.loc[:split_index-1, 'pred_lgb'] = lgb_model.predict(X_train)
  df_result.loc[split_index:, 'pred_lgb'] = lgb_model.predict(X_test)

  df_result.loc[:split_index-1, 'pred_mlp'] = mlp.predict(X_train_scaled)
  df_result.loc[split_index:, 'pred_mlp'] = mlp.predict(X_test_scaled)

  # 예측값 t+1에 위치시키기
  for col in ['pred_mlr', 'pred_svr', 'pred_lgb', 'pred_mlp']:
      df_result[col] = df_result[col].shift(1)

  # 시간 컬럼 복원
  X_test_result = df_result.copy()
  X_test_result.loc[:split_index-1, 'Time'] = time_train.values
  X_test_result.loc[split_index:, 'Time'] = time_test.values
  X_test_result.dropna(subset=['pred_mlr', 'pred_svr', 'pred_lgb', 'pred_mlp'], inplace=True)

  forecasted_df_list.append(X_test_result)

  # 모델 저장
  joblib.dump(mlr, f"{path}/result_of_paper/ann/mlr_forecasting_model_{str(a[i])}.joblib")
  joblib.dump(svr, f"{path}/result_of_paper/ann/svr_forecasting_model_{str(a[i])}.joblib")
  joblib.dump(mlp, f"{path}/result_of_paper/ann/mlp_forecasting_model_{str(a[i])}.joblib")
  lgb_model.booster_.save_model(f"{path}/result_of_paper/ann/lgb_forecasting_model_{str(a[i])}.txt")


# 오차 계산
for site_num, forecasted_df in zip([1,2,4,5,6,7,8], forecasted_df_list):
  for model in ['mlr', 'svr', 'lgb', 'mlp']:
    forecasted_df[f'pred_{model}'] = forecasted_df[f'pred_{model}'].apply(lambda x: max(x, 0))
    forecasted_df[f'error_{model}'] = forecasted_df.apply(
        lambda row: (row[f'pred_{model}'] - row['Power (MW)']),
        axis=1
    )

  result_for_reforecast_df_list.append(forecasted_df)


1
MLR
SVR


In [ ]:
a = [1,2,4,5,6,7,8]
operation_hours_array = [
  ('06:00', '21:30'),
  ('00:00', '23:59'),
  ('00:00', '23:59'),
  ('00:00', '23:59'),
  ('06:00', '21:00'),
  ('06:00', '21:00'),
  ('06:00', '19:00')
]


filtered_df_list = []

for final_result_df, df_index in zip(result_for_reforecast_df_list, a):
  final_result_df.to_csv(f"{path}/result_of_paper/ann/forecast_result_{df_index}_.csv", index=False)

for df,(start_str, end_str) in zip(result_for_reforecast_df_list, operation_hours_array):
  split_index = int(len(df) * (1 - test_size))
  df_copy = df.iloc[split_index:].copy()

  df_copy = df.copy()
  df_copy['Time'] = pd.to_datetime(df_copy['Time'])

  start_time = time.fromisoformat(start_str)
  end_time = time.fromisoformat(end_str)

  df_copy = df_copy[df_copy['Time'].dt.time.between(start_time, end_time)]
  filtered_df_list.append(df_copy)


for result_df, df_index in zip(filtered_df_list, a):
  print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
  print(df_index)

  for forecast_model in ['mlr','svr','lgb', 'mlp']:
    print(f'reforecast model : {forecast_model.upper()}')

    mse = (result_df[f'error_{forecast_model}']** 2).mean()
    mae = result_df[f'error_{forecast_model}'].abs().mean()
    rmse = np.sqrt(mse)

    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
1
reforecast model : MLR
MAE: 1.3432371657455586
MSE: 5.8912607783007305
RMSE: 2.4271919533281108
reforecast model : SVR
MAE: 0.9817740981865034
MSE: 4.857985210537999
RMSE: 2.20408375760496
reforecast model : LGB
MAE: 0.9215383391501577
MSE: 3.488440289056397
RMSE: 1.8677366755130116
reforecast model : MLP
MAE: 1.0758626561357554
MSE: 4.333918637148629
RMSE: 2.0818065801482684
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
2
reforecast model : MLR
MAE: 2.1117283610250213
MSE: 19.3799192124378
RMSE: 4.402262964934944
reforecast model : SVR
MAE: 1.7471183922350109
MSE: 18.918254578422005
RMSE: 4.349511993134633
reforecast model : LGB
MAE: 1.492455995281422
MSE: 12.532524290046915
RMSE: 3.5401305470345177
reforecast model : MLP
MAE: 1.7741521743371806
MSE: 15.026137885748636
RMSE: 3.876

## Reforecast

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor  # ✅ MLP 추가
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
import numpy as np
import math

test_size = test_size  # 기존 전역 변수 유지

features = ['Total solar irradiance (W/m2)',
       'Air temperature  (°C) ', 'Atmosphere (hpa)', 'Relative humidity (%)',
       'Power (MW)', 'hour_sin', 'hour_cos', 'month_sin',
       'month_cos', 'same_time_mean', 'same_time_std', 'same_time_max',
       'same_time_min', 'same_weekday_mean', 'same_weekday_std',
       'same_weekday_max', 'same_weekday_min', 'overall_mean', 'overall_std',
       'overall_max', 'overall_min', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
       'lag_5', 'lag_6', 'lag_7', 'lag_8', 'diff_15min', 'diff_30min',
       'diff_45min', 'diff_60min', 'diff_75min', 'diff_90min', 'diff_105min']

# ✅ 예측 대상 오차 (error) 컬럼명
mlr_target = 'error_mlr'
svr_target = 'error_svr'
lgb_target = 'error_lgb'
mlp_target = 'error_mlp'  # ✅ 추가
targets = [mlr_target, svr_target, lgb_target, mlp_target]  # ✅ 추가됨

# ✅ 각 모델별 재예측에 사용될 feature 목록
forecast_mlr_features = features + ['error_mlr', 'pred_mlr', 'same_time_mean_error_mlr',
                                    'same_time_std_error_mlr',
                                    'same_time_max_error_mlr',
                                    'same_time_min_error_mlr',
                                    'same_weekday_mean_error_mlr',
                                    'same_weekday_std_error_mlr',
                                    'same_weekday_max_error_mlr',
                                    'same_weekday_min_error_mlr',
                                    'overall_mean_error_mlr',
                                    'overall_std_error_mlr',
                                    'overall_max_error_mlr',
                                    'overall_min_error_mlr']

forecast_svr_features = features + ['error_svr', 'pred_svr', 'same_time_mean_error_svr',
                                    'same_time_std_error_svr',
                                    'same_time_max_error_svr',
                                    'same_time_min_error_svr',
                                    'same_weekday_mean_error_svr',
                                    'same_weekday_std_error_svr',
                                    'same_weekday_max_error_svr',
                                    'same_weekday_min_error_svr',
                                    'overall_mean_error_svr',
                                    'overall_std_error_svr',
                                    'overall_max_error_svr',
                                    'overall_min_error_svr']

forecast_lgb_features = features + ['error_lgb', 'pred_lgb', 'same_time_mean_error_lgb',
                                    'same_time_std_error_lgb',
                                    'same_time_max_error_lgb',
                                    'same_time_min_error_lgb',
                                    'same_weekday_mean_error_lgb',
                                    'same_weekday_std_error_lgb',
                                    'same_weekday_max_error_lgb',
                                    'same_weekday_min_error_lgb',
                                    'overall_mean_error_lgb',
                                    'overall_std_error_lgb',
                                    'overall_max_error_lgb',
                                    'overall_min_error_lgb']

forecast_mlp_features = features + ['error_mlp', 'pred_mlp', 'same_time_mean_error_mlp',
                                    'same_time_std_error_mlp',
                                    'same_time_max_error_mlp',
                                    'same_time_min_error_mlp',
                                    'same_weekday_mean_error_mlp',
                                    'same_weekday_std_error_mlp',
                                    'same_weekday_max_error_mlp',
                                    'same_weekday_min_error_mlp',
                                    'overall_mean_error_mlp',
                                    'overall_std_error_mlp',
                                    'overall_max_error_mlp',
                                    'overall_min_error_mlp']

# ✅ 모델별 feature list
features_list = [forecast_mlr_features, forecast_svr_features, forecast_lgb_features, forecast_mlp_features]

# ✅ shift된 오차 타겟명
shifted_mlr_target = 'error_mlr_shifted'
shifted_svr_target = 'error_svr_shifted'
shifted_lgb_target = 'error_lgb_shifted'
shifted_mlp_target = 'error_mlp_shifted'  # ✅ 추가
shifted_targets = [shifted_mlr_target, shifted_svr_target, shifted_lgb_target, shifted_mlp_target]


In [ ]:
import joblib
import numpy as np
from datetime import time
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor  # ✅ 추가
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

final_result_lists = [[], [], [], []]
# result_for_reforecast_df_list 사용
# 모델 리스트 추가
for forecast_model, target, shifted_target, final_result_list, features in zip(
    ['mlr', 'svr', 'lgb', 'mlp'],
    targets,
    shifted_targets,
    final_result_lists,
    features_list):

  print('============================================================================================')
  print(f'forecast model : {forecast_model.upper()}')

  reforecasted_df_list = []
  for df, df_index in zip(result_for_reforecast_df_list, a):
    print('----------------------------------------------------------------------------------------------')
    print(df_index)

    df[shifted_target] = df[target].shift(-1)
    df = df.dropna().reset_index(drop=True)

    features = [col for col in features if col in df.columns]

    X = df[features]
    y = df[shifted_target]
    time_col = df['Time']

    # 시계열 순서 기반 분할
    split_index = int(len(X) * (1 - test_size))

    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
    time_train, time_test = time_col.iloc[:split_index], time_col.iloc[split_index:]

    # 정규화
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 모델 학습
    print("MLR")
    mlr = LinearRegression()
    mlr.fit(X_train_scaled, y_train)

    print("SVR")
    svr = SVR()
    svr.fit(X_train_scaled, y_train)

    print("LightGBM")
    lgb_model = lgb.LGBMRegressor()
    lgb_model.fit(X_train, y_train)

    print("MLP")
    mlp = MLPRegressor(random_state=42, max_iter=1000)
    mlp.fit(X_train_scaled, y_train)

    # 예측 결과를 t+1 위치에 저장
    df_result = X.copy()
    df_result.loc[:split_index-1, 'pred_error_mlr'] = mlr.predict(X_train_scaled)
    df_result.loc[split_index:, 'pred_error_mlr'] = mlr.predict(X_test_scaled)

    df_result.loc[:split_index-1, 'pred_error_svr'] = svr.predict(X_train_scaled)
    df_result.loc[split_index:, 'pred_error_svr'] = svr.predict(X_test_scaled)

    df_result.loc[:split_index-1, 'pred_error_lgb'] = lgb_model.predict(X_train)
    df_result.loc[split_index:, 'pred_error_lgb'] = lgb_model.predict(X_test)

    df_result.loc[:split_index-1, 'pred_error_mlp'] = mlp.predict(X_train_scaled)
    df_result.loc[split_index:, 'pred_error_mlp'] = mlp.predict(X_test_scaled)

    # shift
    for col in ['pred_error_mlr', 'pred_error_svr', 'pred_error_lgb', 'pred_error_mlp']:
        df_result[col] = df_result[col].shift(1)

    # 시간 복원
    X_test_result = df_result.copy()
    X_test_result.loc[:split_index-1, 'Time'] = time_train.values
    X_test_result.loc[split_index:, 'Time'] = time_test.values
    X_test_result.dropna(subset=['pred_error_mlr', 'pred_error_svr', 'pred_error_lgb', 'pred_error_mlp'], inplace=True)

    reforecasted_df_list.append(X_test_result)

    # 모델 저장
    joblib.dump(mlr, f"{path}/result_of_paper/ann/mlr_reforecasting_model_{forecast_model}_forecast_{str(df_index)}.joblib")
    joblib.dump(svr, f"{path}/result_of_paper/ann/svr_reforecasting_model_{forecast_model}_forecast_{str(df_index)}.joblib")
    joblib.dump(mlp, f"{path}/result_of_paper/ann/mlp_reforecasting_model_{forecast_model}_forecast_{str(df_index)}.joblib")
    lgb_model.booster_.save_model(f"{path}/result_of_paper/ann/lgb_reforecasting_model_{forecast_model}_forecast_{str(df_index)}.txt")


  # 재예측 값 계산 및 저장
  for forecasted_df in reforecasted_df_list:
    forecasted_df['reforecasted_PV_mlr'] = np.maximum((forecasted_df[f'pred_{forecast_model}'] - forecasted_df['pred_error_mlr']), 0)
    forecasted_df['reforecasted_PV_svr'] = np.maximum((forecasted_df[f'pred_{forecast_model}'] - forecasted_df['pred_error_svr']), 0)
    forecasted_df['reforecasted_PV_lgb'] = np.maximum((forecasted_df[f'pred_{forecast_model}'] - forecasted_df['pred_error_lgb']), 0)
    forecasted_df['reforecasted_PV_mlp'] = np.maximum((forecasted_df[f'pred_{forecast_model}'] - forecasted_df['pred_error_mlp']), 0)

    final_result_list.append(forecasted_df)


forecast model : MLR
----------------------------------------------------------------------------------------------
1
MLR
SVR
LightGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7913
[LightGBM] [Info] Number of data points in the train set: 63701, number of used features: 35
[LightGBM] [Info] Start training from score 0.076932
MLP
----------------------------------------------------------------------------------------------
2
MLR
SVR
LightGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001421 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is no

FORECAST COPY 잘못됨. EVAL 다시

In [ ]:
operation_hours_array = [
  ('06:00', '21:30'),
  ('00:00', '23:59'),
  ('00:00', '23:59'),
  ('00:00', '23:59'),
  ('06:00', '21:00'),
  ('06:00', '21:00'),
  ('06:00', '19:00')
]
for forecast_model, final_result_list in zip(['mlr','svr','lgb','mlp'], final_result_lists):
  print('================================================================================================================')
  print(f'forecast model : {forecast_model.upper()}')

  for final_result_df, df_index in zip(final_result_list, a):
    final_result_df.to_csv(f"{path}/result_of_paper/ann/reforecast_result_{forecast_model}_{df_index}_.csv", index=False)

  filtered_df_list = []
  for df,(start_str, end_str) in zip(final_result_list, operation_hours_array):
    split_index = int(len(df) * (1 - test_size))
    df_copy = df.iloc[split_index:].copy()
    df_copy['Time'] = pd.to_datetime(df_copy['Time'])

    start_time = time.fromisoformat(start_str)
    end_time = time.fromisoformat(end_str)

    df_copy = df_copy[df_copy['Time'].dt.time.between(start_time, end_time)]
    filtered_df_list.append(df)

  for result_df, df_index in zip(filtered_df_list, a):
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print(df_index)

    for reforecast_model in ['mlr','svr','lgb','mlp']:
      print(f'reforecast model : {reforecast_model.upper()}')

      mse = ((result_df['Power (MW)'] - result_df[f'reforecasted_PV_{reforecast_model}']) ** 2).mean()
      mae = (result_df['Power (MW)'] - result_df[f'reforecasted_PV_{reforecast_model}']).abs().mean()
      rmse = np.sqrt(mse)

      print(f"MAE: {mae}")
      print(f"MSE: {mse}")
      print(f"RMSE: {rmse}")


forecast model : MLR
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
1
reforecast model : MLR
MAE: 0.8975295341206315
MSE: 3.8470737453493378
RMSE: 1.9613958665576252
reforecast model : SVR
MAE: 0.6176318295195427
MSE: 3.1346791411625903
RMSE: 1.770502510916771
reforecast model : LGB
MAE: 0.6020785303912266
MSE: 2.2764821545569243
RMSE: 1.5088015623523605
reforecast model : MLP
MAE: 0.7026405079438782
MSE: 2.6724876364050503
RMSE: 1.6347744909941095
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
2
reforecast model : MLR
MAE: 2.1030411988271536
MSE: 19.284996582570823
RMSE: 4.391468613410646
reforecast model : SVR
MAE: 1.5581672910427222
MSE: 16.751699189451227
RMSE: 4.092883969702932
reforecast model : LGB
MAE: 1.5004333711762992
MSE: 12.151586509777136
RMSE: 3.485912579193164
reforecast model : MLP
MAE: 1.7771893428043826
MSE: 14.

FORECAST EVAL

In [ ]:
a = [1,2,4,5,6,7,8]
operation_hours_array = [
  ('06:00', '21:30'),
  ('00:00', '23:59'),
  ('00:00', '23:59'),
  ('00:00', '23:59'),
  ('06:00', '21:00'),
  ('06:00', '21:00'),
  ('06:00', '19:00')
]


filtered_df_list = []
forecast_df_list = []
for final_result_df, df_index in zip(result_for_reforecast_df_list, a):
  forecast_df_list.append(pd.read_csv(f"{path}/result_of_paper/ann/forecast_result_{df_index}_.csv"))

for df,(start_str, end_str) in zip(forecast_df_list, operation_hours_array):
  split_index = int(len(df) * (1 - test_size))
  df_copy = df.iloc[split_index:].copy()
  df_copy['Time'] = pd.to_datetime(df_copy['Time'])

  start_time = time.fromisoformat(start_str)
  end_time = time.fromisoformat(end_str)

  df_copy = df_copy[df_copy['Time'].dt.time.between(start_time, end_time)]
  filtered_df_list.append(df_copy)


for result_df, df_index in zip(filtered_df_list, a):
  print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
  print(df_index)

  for forecast_model in ['mlr','svr','lgb', 'mlp']:
    print(f'reforecast model : {forecast_model.upper()}')

    mse = (result_df[f'error_{forecast_model}']** 2).mean()
    mae = result_df[f'error_{forecast_model}'].abs().mean()
    rmse = np.sqrt(mse)

    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
1
reforecast model : MLR
MAE: 1.2143396211577149
MSE: 4.673142000224163
RMSE: 2.1617451284145788
reforecast model : SVR
MAE: 0.8225092692725493
MSE: 3.5781648873501295
RMSE: 1.8916037870944669
reforecast model : LGB
MAE: 0.8236550088847088
MSE: 3.47332759837725
RMSE: 1.863686561194572
reforecast model : MLP
MAE: 0.8988507370493319
MSE: 3.51095476038574
RMSE: 1.8737541888907787
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
2
reforecast model : MLR
MAE: 2.16632267625182
MSE: 22.594579094125255
RMSE: 4.753375547347932
reforecast model : SVR
MAE: 2.1035599790892383
MSE: 25.05637192497996
RMSE: 5.005634018281796
reforecast model : LGB
MAE: 1.7016949229849754
MSE: 19.781507967872454
RMSE: 4.447640719288424
reforecast model : MLP
MAE: 2.0587794278983385
MSE: 20.159160804664634
RMSE: 4.48989